In [ ]:
import pandas as pd
import random
import numpy as np

# Carregar os dados das planilhas
escala_df = pd.read_excel('/content/Escala_Final 1 (1).xlsx')
disponibilidade_df = pd.read_excel('Disponibilidade_2025.xlsx')
inspetores_df = pd.read_excel('Inspetores_2025.xlsx')
preferencia_df = pd.read_excel('Preferencia_2025.xlsx')

# Converter as colunas "Semana" para datetime
disponibilidade_df['Semana'] = pd.to_datetime(disponibilidade_df['Semana'], format='%d/%m/%Y')
escala_df['Semana'] = pd.to_datetime(escala_df['Semana'], format='%d/%m/%Y', errors='coerce')

# Cria um dicionário de preferências:
# chave: nome do inspetor; valor: nome do parceiro preferido
preferencias = dict(zip(preferencia_df['Nome'], preferencia_df['Preferência']))

def preencher_escala_corrigido_v21(escala, disponibilidade, inspetores, preferencias,
                                   max_escalas=7, priorizar_menos_escala=True):
    """
    Preenche a escala obedecendo às regras:
      - Respeitar a disponibilidade dos inspetores para cada semana.
      - Selecionar apenas os inspetores capacitados para o tipo de empresa.
      - Garantir que um inspetor não seja alocado em mais de uma empresa na mesma semana.
      - Limitar o número total de escalas por inspetor (max_escalas).
      - Evitar escalas consecutivas (exceto para a região 'C').
      - Priorizar o parceiro preferido, se disponível.
      - Garantir que pelo menos um dos inspetores seja líder.

      Apenas os tipos de empresa permitidos são processados:
        'Sólidos', 'Medicamentos', 'Estéreis', 'Insumos', 'Biológicos'

    A novidade nesta versão é que, ao selecionar os inspetores (tanto o 1º quanto o 2º),
    o algoritmo prioriza aqueles que já possuem menos dias escalados (ou seja, com
    menor número de atribuições no período), evitando que inspetores que já preencheram
    muitas semanas fiquem sempre à frente.

    Parâmetros:
      - priorizar_menos_escala: se True, dentre os candidatos elegíveis, é feita a seleção
        preferencialmente entre aqueles com menor número de escalas já marcadas.
    """
    # Converter datas para datetime, se necessário
    disponibilidade['Semana'] = pd.to_datetime(disponibilidade['Semana'], format='%d/%m/%Y', errors='coerce')
    escala['Semana'] = pd.to_datetime(escala['Semana'], format='%d/%m/%Y', errors='coerce')

    alocacao_lideres = []
    alocacao_inspetores = []
    # Dicionário que guarda os inspetores já alocados por semana (chave: semana no formato 'dd/mm/aaaa')
    inspetores_alocados_por_semana = {}
    # Histórico de escalas por inspetor (chave: nome, valor: lista de semanas escaladas)
    inspetor_historico = {}

    # Lista com os tipos permitidos
    tipos_validos = ['Sólidos', 'Medicamentos', 'Estéreis', 'Insumos', 'Biológicos']

    for i, row in escala.iterrows():
        semana_data = row['Semana']
        tipo_empresa = row['Tipo']
        regiao = row['Região']

        # Se faltar data ou tipo, deixa em branco
        if pd.isna(semana_data) or pd.isna(tipo_empresa):
            alocacao_lideres.append('')
            alocacao_inspetores.append('')
            continue

        # Se o tipo de empresa não estiver entre os tipos permitidos, pula a alocação
        if tipo_empresa not in tipos_validos:
            alocacao_lideres.append('')
            alocacao_inspetores.append('')
            continue

        # Seleciona os inspetores disponíveis para a semana
        inspetores_disponiveis = disponibilidade[disponibilidade['Semana'] == semana_data]['Inspetor'].unique()

        # Filtrar inspetores capacitados conforme o tipo de empresa
        if tipo_empresa == 'Sólidos':
            # Sólidos correspondem a Medicamentos
            inspetores_capacitados = inspetores[inspetores['Medicamentos'] != 'Não capacitado']
        elif tipo_empresa == 'Medicamentos':
            inspetores_capacitados = inspetores[inspetores['Medicamentos'] != 'Não capacitado']
        elif tipo_empresa == 'Estéreis':
            inspetores_capacitados = inspetores[inspetores['Estéreis'] != 'Não capacitado']
        elif tipo_empresa == 'Insumos':
            inspetores_capacitados = inspetores[inspetores['Insumos'] != 'Não capacitado']
        elif tipo_empresa == 'Biológicos':
            inspetores_capacitados = inspetores[inspetores['Biológicos'] != 'Não capacitado']

        # Filtrar os inspetores que estão disponíveis e capacitados
        inspetores_qualificados = inspetores_capacitados[inspetores_capacitados['Inspetor'].isin(inspetores_disponiveis)]
        if inspetores_qualificados.empty:
            alocacao_lideres.append('')
            alocacao_inspetores.append('')
            continue

        # Separar os que são líderes e os que não são
        lideres_lista = inspetores_qualificados[inspetores_qualificados['Liderança'] == 'Líder']['Inspetor'].tolist()
        nao_lideres_lista = inspetores_qualificados[inspetores_qualificados['Liderança'] == 'Não Líder']['Inspetor'].tolist()

        # Formata a semana para string (para uso no histórico e na verificação de alocação)
        semana_str = semana_data.strftime('%d/%m/%Y')
        semana_anterior = (semana_data - pd.Timedelta(days=7)).strftime('%d/%m/%Y')

        # Filtra os candidatos que:
        #  - Ainda não atingiram o limite de escalas
        #  - Não foram alocados para outra empresa nessa mesma semana
        candidatos = [
            insp for insp in (lideres_lista + nao_lideres_lista)
            if len(inspetor_historico.get(insp, [])) < max_escalas and
               insp not in inspetores_alocados_por_semana.get(semana_str, set())
        ]

        # Se a região não for 'C', evita escalas consecutivas (quem trabalhou na semana anterior)
        if regiao != 'C':
            candidatos = [insp for insp in candidatos if semana_anterior not in inspetor_historico.get(insp, [])]

        # Caso reste menos de 2 candidatos, relaxa a restrição de escalas consecutivas (mantendo a verificação da mesma semana)
        if len(candidatos) < 2:
            candidatos = [
                insp for insp in (lideres_lista + nao_lideres_lista)
                if len(inspetor_historico.get(insp, [])) < max_escalas and
                   insp not in inspetores_alocados_por_semana.get(semana_str, set())
            ]

        if len(candidatos) < 2:
            inspetor_1 = ''
            inspetor_2 = ''
        else:
            # Seleciona o inspetor 1 entre os candidatos, priorizando aqueles com menos dias já escalados
            if priorizar_menos_escala:
                min_escalas = min(len(inspetor_historico.get(insp, [])) for insp in candidatos)
                candidatos_priorizados = [insp for insp in candidatos if len(inspetor_historico.get(insp, [])) == min_escalas]
                inspetor_1 = random.choice(candidatos_priorizados)
            else:
                inspetor_1 = random.choice(candidatos)

            # Após selecionar o inspetor 1, constrói a lista de candidatos para o inspetor 2
            remaining_candidates = [insp for insp in candidatos if insp != inspetor_1]

            # Verifica se o inspetor 1 tem um parceiro preferido disponível
            parceiro_preferido = preferencias.get(inspetor_1, None)
            if parceiro_preferido and parceiro_preferido in remaining_candidates:
                # Se a priorização estiver ativa, verifica se o parceiro está entre os com menor número de escalas
                if priorizar_menos_escala:
                    min_remaining = min(len(inspetor_historico.get(insp, [])) for insp in remaining_candidates)
                    if len(inspetor_historico.get(parceiro_preferido, [])) == min_remaining:
                        inspetor_2 = parceiro_preferido
                    else:
                        candidatos2_priorizados = [insp for insp in remaining_candidates if len(inspetor_historico.get(insp, [])) == min_remaining]
                        inspetor_2 = random.choice(candidatos2_priorizados)
                else:
                    inspetor_2 = parceiro_preferido
            else:
                if remaining_candidates:
                    if priorizar_menos_escala:
                        min_remaining = min(len(inspetor_historico.get(insp, [])) for insp in remaining_candidates)
                        candidatos2_priorizados = [insp for insp in remaining_candidates if len(inspetor_historico.get(insp, [])) == min_remaining]
                        inspetor_2 = random.choice(candidatos2_priorizados)
                    else:
                        inspetor_2 = random.choice(remaining_candidates)
                else:
                    inspetor_2 = ''

            # Garante que pelo menos um dos inspetores seja líder
            if not (inspetor_1 in lideres_lista or inspetor_2 in lideres_lista):
                possiveis_lideres = [
                    insp for insp in lideres_lista
                    if len(inspetor_historico.get(insp, [])) < max_escalas and
                       insp not in inspetores_alocados_por_semana.get(semana_str, set())
                ]
                if possiveis_lideres:
                    leader_sel = random.choice(possiveis_lideres)
                    inspetor_1 = leader_sel

            # Atualiza o histórico dos inspetores (somente se houver inspetor selecionado)
            if inspetor_1:
                inspetor_historico.setdefault(inspetor_1, []).append(semana_str)
            if inspetor_2:
                inspetor_historico.setdefault(inspetor_2, []).append(semana_str)
            # Atualiza a lista de inspetores já alocados na semana
            inspetores_alocados_por_semana.setdefault(semana_str, set()).update([inspetor_1, inspetor_2])

        alocacao_lideres.append(inspetor_1)
        alocacao_inspetores.append(inspetor_2)

    escala['Inspetor 1'] = alocacao_lideres
    escala['Inspetor 2'] = alocacao_inspetores
    return escala

# Exemplo de execução:
# Aqui, a priorização é feita de forma que os inspetores com menos dias já escalados sejam escolhidos primeiro.
escala_final = preencher_escala_corrigido_v21(
    escala_df.copy(),
    disponibilidade_df,
    inspetores_df,
    preferencias,
    max_escalas=7,
    priorizar_menos_escala=True  # Altere para False se preferir a escolha puramente aleatória entre os candidatos
)

# Formata a coluna "Semana" e salva o resultado em Excel
escala_final['Semana'] = escala_final['Semana'].dt.strftime('%d/%m/%Y')
escala_final.to_excel('Escala_V21.xlsx', index=False)

# Exibe a frequência de escalas por inspetor para conferência
frequencia_escala = pd.concat([escala_final['Inspetor 1'], escala_final['Inspetor 2']]).value_counts()

# Exibe o resultado em formato de tabela markdown (se preferir, pode usar to_string ou to_markdown)
print(escala_final.to_markdown())
print('================== Frequência ==================')
print(frequencia_escala)